In [10]:
import pandas as pd 
import datetime 
from datetime import timezone
#from datetime import datetime
#this is all old code that should be deleted eventually 
#DO NOT RUN 
pw = pd.read_parquet('part-00000-49a36603-9035-47f4-b73e-eae8d28aa10a-c000.gz.parquet')
pw.head()
outage = pw[pw['is_powered']]
outage = outage[['time', 'location_latitude', 'location_longitude']]
outage.head()

In [16]:
outage['time'].values[0]

In [12]:
jul_1 = outage[outage['time'] < datetime.datetime(2018, 7, 1, 0, 2)]
jul_1['time'] = jul_1['time'].apply(lambda x: x.replace(tzinfo=timezone.utc).timestamp())
jul_1.head()

In [15]:
type(jul_1['time'].values[0])

In [ ]:
# expanded = pw.join(pd.DataFrame(pw_time['time'].values.tolist(), columns=['year','month', 'day', 'hour', 'min', 'sec', 'sec1', 'sec2', 'sec3'], index=pw_time.index))
# expanded = expanded[['powered_longitude', 'powered_latitude', 'outage_longitude', 'outage_latitude', 'year','month', 'day', 'hour', 'min', 'sec']]
# expanded

In [ ]:
#this is running DBSCAN on all outages (not just the transition points)
pw_cluster = StandardScaler().fit_transform(jul_1)
db = DBSCAN(eps=.65, algorithm='ball_tree').fit(pw_cluster)
labels = db.labels_
n_noise_ = list(labels).count(-1)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_, n_clusters_ 



In [ ]:
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels,
#                                            average_method='arithmetic'))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(pw_cluster, labels))

In [ ]:
#check with raw data of outages and times that they went out and then cluster based on outages. then see which ones seem to be the most reliably clustering 
jul_1['labels'] = labels
jul_1.head()

In [ ]:
jul_1_0 = jul_1[jul_1['labels'] == 0]
jul_1_1 = jul_1[jul_1['labels'] == 1]
jul_1_2 = jul_1[jul_1['labels'] == 2]
jul_1_3 = jul_1[jul_1['labels'] == 3]
jul_1_4 = jul_1[jul_1['labels'] == 4]
jul_1_5 = jul_1[jul_1['labels'] == 5]
jul_1_6 = jul_1[jul_1['labels'] == 6]
jul_1_7 = jul_1[jul_1['labels'] == 7]
jul_1_8 = jul_1[jul_1['labels'] == 8]
jul_1_9 = jul_1[jul_1['labels'] == 9]
jul_1_unlabeled = jul_1[jul_1['labels'] == -1]

In [ ]:
#to examine a difference in classification, let's zoom in at the data in the top cluster of this plot 
#print out the distance in time between the points 
a_time = jul_1[jul_1['location_latitude'] > 5.66]['time'].values[0]
for i in jul_1[jul_1['location_latitude'] > 5.66]['time'].values[1:]: 
    print(abs(a_time - i))

In [ ]:
#now let's compare that with all the points in cluster 2
time1 = jul_1[jul_1['labels'] == 2]['time'].values[0]
for i in jul_1[jul_1['labels'] == 2]['time'].values[1:]: 
    print((time1 - i))

In [ ]:
plt.scatter(pw_time['powered_longitude'], pw_time['powered_latitude'], c='b', label='powered')
plt.scatter(pw_time['outage_longitude'], pw_time['outage_latitude'], c='r', label='outage')
plt.title('Powered Sensors vs Outages')
plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')

In [ ]:
#Kneedle analysis to find optimal epsilon 
ns = 2
nbrs = NearestNeighbors(n_neighbors=ns).fit(X)
distances, indices = nbrs.kneighbors(X)
y = np.array(sorted(distances[:,ns-1], reverse=True))
x= np.arange(1, len(X)+1)
plt.plot(list(range(1,len(X)+1)), y)

In [ ]:
#more knee 
from kneed import KneeLocator
kneedle = KneeLocator(x, y, S=.001, curve='convex', direction='decreasing')
kneedle.plot_knee_normalized()
kneedle.plot_knee()
print(round(kneedle.knee, 3))

In [ ]:
#implementing gradient descent: 
def average_squared_loss(y, y_hat):
    """
    Returns the averge squared loss for observations y and predictions y_hat.

    Keyword arguments:
    y -- the vector of true values y
    y_hat -- the vector of predicted values y_hat
    """
    return np.mean(np.square(y - y_hat))

def dt1(x, y, theta):
    """
    Compute the numerical value of the partial of l2 loss with respect to theta_1

    Keyword arguments:
    x -- the vector of all x values
    y -- the vector of all y values
    theta -- the vector of values theta
    """
    return -2*sum(y*x - theta[0]*x*x - x*np.sin(x*theta[1]))/len(x)
    
def dt2(x, y, theta):
    """
    Compute the numerical value of the partial of l2 loss with respect to theta_2

    Keyword arguments:
    x -- the vector of all x values
    y -- the vector of all y values
    theta -- the vector of values theta
    """
    chain_rule = x*np.cos(x*theta[1])
    return -2*sum(y*chain_rule - x*chain_rule*theta[0] - chain_rule*np.sin(x*theta[1]))/len(x)
    
# This function calls dt1 and dt2 and returns the gradient dt. It is already implemented for you.
def dt(x, y, theta):
    """
    Returns the gradient of l2 loss with respect to vector theta

    Keyword arguments:
    x -- the vector of values x
    y -- the vector of values y
    theta -- the vector of values theta
    """
    return np.array([dt1(x,y,theta), dt2(x,y,theta)])


# Then use gradient descent to find the optimal 
def init_t():
    """Creates an initial theta [0, 0] of shape (2,) as a starting point for gradient descent"""
    return np.zeros((2,))

def grad_desc(x, y, theta, num_iter=20, alpha=0.1):
    """
    Run gradient descent update for a finite number of iterations and static learning rate

    Keyword arguments:
    x -- the vector of values x
    y -- the vector of values y
    theta -- the vector of values theta to use at first iteration
    num_iter -- the max number of iterations
    alpha -- the learning rate (also called the step size)
    
    Return:
    theta -- the optimal value of theta after num_iter of gradient descent
    theta_history -- the series of theta values over each iteration of gradient descent
    loss_history -- the series of loss values over each iteration of gradient descent
    """
    theta_history = []
    loss_history = []
    
    for i in range(num_iter): 
        theta_history.append(theta)
        loss_history.append(average_squared_loss(y, [theta[0]]*len(y)))
        theta = theta - alpha*(dt(x, y, theta))
    
    return theta, theta_history, loss_history

t = init_t()
t_est, ts, loss = grad_desc(x, y, t, num_iter=20, alpha=0.1)
print(t_est)

Now, this is trash code from STDBSCAN (it's basically trying STDBSCAN on 2 days worth of data to verify that it's clustering well)

In [ ]:
#now try it on 2 days of data to do the analysis that I was performing in evaluating_clustering_algorithm
days = outages[outages['outage_time'] <= min(outages['outage_time'])+172800]
days = pd.DataFrame(test_time(days))
days.head()

In [ ]:
days = days.rename(columns={'cluster': 'labels'})
day_a = days[days['labels'] == 0]
day_b = days[days['labels'] == 1]
day_c = days[days['labels'] == 2]
day_d = days[days['labels'] == 3]
day_e = days[days['labels'] == 4]
day_f = days[days['labels'] == 5]
unlabeled = days[days['labels'] == -1]
plt.figure(figsize=(10,10))
plt.scatter(unlabeled['location_longitude'], unlabeled['location_latitude'], c='y',label='noise')
plt.scatter(day_b['location_longitude'], day_b['location_latitude'], label='cluster 1')
plt.scatter(day_c['location_longitude'], day_c['location_latitude'], label='cluster 2')
plt.scatter(day_d['location_longitude'], day_d['location_latitude'], label='cluster 3')
plt.scatter(day_e['location_longitude'], day_e['location_latitude'], label='cluster 4')
#plt.scatter(day_f['location_longitude'], day_f['location_latitude'], label='cluster 5')
plt.title('STDBSCAN Clustered Outages from 7/1/18 - 7/2/18')
plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')
# plt.xlim(left, right)
# plt.ylim(top, bottom)

In [ ]:
def find_range(lst):
    return max(lst) - min(lst)

#days['time'] = days['time'].apply(lambda x: int(x.replace(tzinfo=timezone.utc).timestamp()))
days_clust = pd.DataFrame(days.groupby(['labels'])['time'].apply(lambda x: x.tolist())).rename(columns={'time': 'outage_times'})
days_clust['latitude'] = days.groupby(['labels'])['location_latitude'].apply(lambda x: x.tolist()).values
days_clust['longitude'] = days.groupby(['labels'])['location_longitude'].apply(lambda x: x.tolist()).values
days_clust = days_clust.iloc[1:]

days_clust['time_range'] = (np.vectorize(find_range)(days_clust['outage_times']))
days_clust['lat_range'] = (np.vectorize(find_range)(days_clust['latitude']))
days_clust['long_range'] = (np.vectorize(find_range)(days_clust['longitude']))

days_clust.head()

In [ ]:
sns.distplot(days_clust['outage_times'].values[3], label='cluster_4')
sns.distplot([1530449581, 1530449581, 1530449582], label='cluster_3')
sns.distplot(days_clust['outage_times'].values[1], label='cluster_2')
sns.distplot(days_clust['outage_times'].values[0], label='cluster_1')
plt.legend()
plt.title('STDBSCAN Clustering Distributions')